<a href="https://colab.research.google.com/github/MandyZhangxy/Data_Mining-Infant-Mortality/blob/master/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Movie Recommendation Project**
In this project, I will use an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies in [MovieLens small dataset](https://grouplens.org/datasets/movielens/latest/)

## **Running Pyspark in Colab**

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

## **Setting up environment**

In [5]:
%matplotlib inline
import os

import numpy as np
import pandas as pd
import seaborn as sns

# from nba_utils import draw_3pt_piechart,plot_shot_chart

from IPython.core.display import display, HTML
from IPython.core.magic import register_cell_magic, register_line_cell_magic, register_line_magic
from matplotlib import pyplot as plt
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import array, col, count, mean, sum, udf, when
from pyspark.sql.types import DoubleType, IntegerType, StringType, Row
from pyspark.sql.functions import sum, col, udf

import warnings
warnings.filterwarnings("ignore")

sns.set_style("white")
sns.set_color_codes()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
!ls

sample_data  spark-2.4.5-bin-hadoop2.7	spark-2.4.5-bin-hadoop2.7.tgz


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### **Data ETL and Data Exploration**

In [8]:
from google.colab import files
files.upload()

movies_df = spark.read.load("movies.csv", format="csv",header = True)
ratings_df = spark.read.load("ratings.csv", format = "csv", header = True)
links_df = spark.read.load("ratings.csv", format = "csv", header = True)
tags_df = spark.read.load("tags.csv", format = "csv", header = True)

Saving links.csv to links.csv
Saving movies.csv to movies.csv
Saving ratings.csv to ratings.csv
Saving tags.csv to tags.csv


In [9]:
!ls

links.csv   ratings.csv  spark-2.4.5-bin-hadoop2.7	tags.csv
movies.csv  sample_data  spark-2.4.5-bin-hadoop2.7.tgz


In [10]:
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [18]:
ratings_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [20]:
links_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [21]:
tags_df.show(5)

+------+-------+---------------+----------+
|userId|movieId|            tag| timestamp|
+------+-------+---------------+----------+
|     2|  60756|          funny|1445714994|
|     2|  60756|Highly quotable|1445714996|
|     2|  60756|   will ferrell|1445714992|
|     2|  89774|   Boxing story|1445715207|
|     2|  89774|            MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows



In [24]:
ratings_df.groupBy("userID").count().show()

+------+-----+
|userID|count|
+------+-----+
|   296|   27|
|   467|   22|
|   125|  360|
|   451|   34|
|     7|  152|
|    51|  359|
|   124|   50|
|   447|   78|
|   591|   54|
|   307|  975|
|   475|  155|
|   574|   23|
|   169|  269|
|   205|   27|
|   334|  154|
|   544|   22|
|   577|  161|
|   581|   40|
|   272|   31|
|   442|   20|
+------+-----+
only showing top 20 rows



In [25]:
tmp1 = ratings_df.groupBy("userID").count().toPandas()['count'].min()
tmp2 = ratings_df.groupBy("movieId").count().toPandas()['count'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1


In [38]:
tmp1 = ratings_df.groupBy("movieId").count().toPandas()['count']==1
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1.sum(), tmp2))

3446 out of 9724 movies are rated by only one user


## **Spark SQL and OLAP**

### Register the DataFrame as a local temporary view

In [0]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

### The number of Users


In [61]:
spark.sql(
"select count(distinct userId) as total_users from ratings"
).show()

+-----------+
|total_users|
+-----------+
|        610|
+-----------+



### The number of Movies

In [62]:
spark.sql(
    "select count(distinct movieId) as total_movies from movies"

).show()

+------------+
|total_movies|
+------------+
|        9742|
+------------+



### numbers of movies that are rated by users:

In [65]:
spark.sql(
    '''
    with t as 
    (select m.movieId, title, genres,userId, rating from 
    movies m left join ratings r
    on m.movieId = r.movieId)

    select count(distinct movieId) 
    from t
    where rating is not null
    '''
).show()

+-----------------------+
|count(DISTINCT movieId)|
+-----------------------+
|                   9724|
+-----------------------+



### Movies not rated before:

In [70]:
spark.sql(
    '''
    with t as 
    (select m.movieId, title, genres,userId, rating from 
    movies m left join ratings r
    on m.movieId = r.movieId)

    select movieId, title,genres
    from t
    where rating is null
    group by 1,2,3
    '''
).show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|   4194|I Know Where I'm ...|   Drama|Romance|War|
|   1076|Innocents, The (1...|Drama|Horror|Thri...|
|  30892|In the Realms of ...|Animation|Documen...|
|  26085|Mutiny on the Bou...|Adventure|Drama|R...|
|   5721|  Chosen, The (1981)|               Drama|
|  32160|Twentieth Century...|              Comedy|
|   2939|      Niagara (1953)|      Drama|Thriller|
|  25855|Roaring Twenties,...|Crime|Drama|Thriller|
|  32371|Call Northside 77...|Crime|Drama|Film-...|
|   6849|      Scrooge (1970)|Drama|Fantasy|Mus...|
|   3338|For All Mankind (...|         Documentary|
|  85565|  Chalet Girl (2011)|      Comedy|Romance|
|   7792|Parallax View, Th...|            Thriller|
|   6668|Road Home, The (W...|       Drama|Romance|
|   3456|Color of Paradise...|               Drama|
|   7020|        Proof (1991)|Comedy|Drama|Romance|
|   8765|Thi

### List All Movie Genres

In [74]:
spark.sql(
    """
    select movie_genres from movies
    lateral view explode(split(genres, '[|]')) as movie_genres 
    where movie_genres <> "(no genres listed)"
    group by 1
    order by 1
    """
).show()

+------------+
|movie_genres|
+------------+
|      Action|
|   Adventure|
|   Animation|
|    Children|
|      Comedy|
|       Crime|
| Documentary|
|       Drama|
|     Fantasy|
|   Film-Noir|
|      Horror|
|        IMAX|
|     Musical|
|     Mystery|
|     Romance|
|      Sci-Fi|
|    Thriller|
|         War|
|     Western|
+------------+



### Count Movie for Each Category

In [76]:
spark.sql(
    """
select movie_genres,count(*) as total_movies from movies
lateral view explode(split(genres,'[|]')) as movie_genres
group by 1
""").show()

+------------------+------------+
|      movie_genres|total_movies|
+------------------+------------+
|             Crime|        1199|
|           Romance|        1596|
|          Thriller|        1894|
|         Adventure|        1263|
|             Drama|        4361|
|               War|         382|
|       Documentary|         440|
|           Fantasy|         779|
|           Mystery|         573|
|           Musical|         334|
|         Animation|         611|
|         Film-Noir|          87|
|(no genres listed)|          34|
|              IMAX|         158|
|            Horror|         978|
|           Western|         167|
|            Comedy|        3756|
|          Children|         664|
|            Action|        1828|
|            Sci-Fi|         980|
+------------------+------------+



### List all movie names in each category

In [77]:
spark.sql(
    """
    select t1.movie_genres, concat_ws("|",collect_set(t1.title)) as list_of_movies
    from
    (
    select title,movie_genres from movies
    lateral view explode(split(genres, '[|]')) as movie_genres
    group by 1,2
    ) t1
    group by 1    
    """
).show()

+------------------+--------------------+
|      movie_genres|      list_of_movies|
+------------------+--------------------+
|             Crime|Stealing Rembrand...|
|           Romance|Vampire in Brookl...|
|          Thriller|Element of Crime,...|
|         Adventure|Ice Age: Collisio...|
|             Drama|Airport '77 (1977...|
|               War|General, The (192...|
|       Documentary|The Barkley Marat...|
|           Fantasy|Masters of the Un...|
|           Mystery|Before and After ...|
|           Musical|U2: Rattle and Hu...|
|         Animation|Ice Age: Collisio...|
|         Film-Noir|Rififi (Du rififi...|
|(no genres listed)|T2 3-D: Battle Ac...|
|              IMAX|Harry Potter and ...|
|            Horror|Sweeney Todd (200...|
|           Western|Man Who Shot Libe...|
|            Comedy|Hysteria (2011)|H...|
|          Children|Ice Age: Collisio...|
|            Action|Stealing Rembrand...|
|            Sci-Fi|Push (2009)|SORI:...|
+------------------+--------------

## **Spark ALS based approach for training model**
using an Spark ML to predict the ratings

ALS (or matrix factorization algorithms) can solve three major problem is user-item matrix:
* **Popularity bias**: refers to system recommends the movies with the most interactions without any personalization.
* **Item cold-start problem**: refers to when movies added to the catalogue have either none or very little interactions while recommender rely on the movie’s interactions to make recommendations.
* **Scalability issue**: refers to lack of the ability to scale to much larger sets of data when more and more users and movies added into our database.


matrix factorization algorithms work by decomposing the user-item interaction matrix into the product of two lower dimensionality rectangular matrices. One matrix can be seen as the user matrix where rows represent users and columns are latent factors. The other matrix is the item matrix where rows are latent factors and columns represent items.

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml.tuning import TrainValidationSplit,ParamGridBuilder

In [35]:
ratings_df.head(5)

[Row(userId='1', movieId='1', rating='4.0', timestamp='964982703'),
 Row(userId='1', movieId='3', rating='4.0', timestamp='964981247'),
 Row(userId='1', movieId='6', rating='4.0', timestamp='964982224'),
 Row(userId='1', movieId='47', rating='5.0', timestamp='964983815'),
 Row(userId='1', movieId='50', rating='5.0', timestamp='964982931')]

In [40]:
ratings_df.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: string (nullable = true)



In [0]:
# convert data type in each column
ratings_df=ratings_df.withColumn("rating",ratings_df["rating"].cast("float").alias("rating"))
ratings_df=ratings_df.withColumn("userId",ratings_df["userId"].cast("float").alias("userId"))
ratings_df=ratings_df.withColumn("movieId",ratings_df["movieId"].cast("float").alias("movieId"))

In [56]:
ratings_df.printSchema()

root
 |-- userId: float (nullable = true)
 |-- movieId: float (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: string (nullable = true)



In [0]:
# split into traning and testing dataset 
(training, test) = ratings_df.randomSplit([0.8, 0.2],seed = 1)

In [0]:
# Create ALS model
als = ALS(userCol = "userId", itemCol="movieId", ratingCol="rating",coldStartStrategy="drop", nonnegative=True)

In [0]:
# Tune model using ParamGridBuilder
param_grid = ParamGridBuilder().addGrid(als.rank, [8,10,12]).addGrid(als.maxIter, [10,13,16]).addGrid(als.regParam,[0.1,0.15,0.2]).build()


In [0]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [0]:
# build cross validation using TranValidationSplit:
tvs = TrainValidationSplit(
    estimator = als,
    estimatorParamMaps=param_grid,
    evaluator = evaluator
)

In [0]:
# Fit ALS model to training data
model = tvs.fit(training)

In [0]:
# Extract best model from the tuning exercise using ParamGridBuilder
best_model = model.bestModel

In [0]:
# Generate predictions and evaluate using RMSE
predictions = best_model.transform(test)
rmse = evaluator.evaluate(predictions)

In [64]:
# Print evaluation metrics and model paramters
print("RMSE = " + str(rmse))
print("**Best Model**")
print(" Rank:", best_model.rank)
print(" Max Iteration:", best_model._java_obj.parent().getMaxIter())
print(" Regularization Prameter:", best_model._java_obj.parent().getRegParam())

RMSE = 0.8672694752529748
**Best Model**
 Rank: 12
 Max Iteration: 16
 Regularization Prameter: 0.15


In [66]:
predictions.sort("userId","rating").show()

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|   1.0| 2253.0|   2.0|964981775|  2.922011|
|   1.0| 1580.0|   3.0|964981125| 4.0236874|
|   1.0| 2093.0|   3.0|964981710| 3.8349037|
|   1.0| 2528.0|   3.0|964982328| 3.8941875|
|   1.0| 2012.0|   4.0|964984176| 4.2039013|
|   1.0|  441.0|   4.0|964980868| 4.5997343|
|   1.0| 1920.0|   4.0|964981780| 3.6132016|
|   1.0|  590.0|   4.0|964982546| 4.3389945|
|   1.0| 2000.0|   4.0|964982211| 4.5627317|
|   1.0| 2174.0|   4.0|964981680| 4.2716885|
|   1.0| 2143.0|   4.0|964981725| 3.5278656|
|   1.0|  356.0|   4.0|964980962|  4.999717|
|   1.0| 2193.0|   4.0|964981710|  4.021544|
|   1.0| 2105.0|   4.0|964981725| 3.7885368|
|   1.0| 2268.0|   4.0|964982989|  4.365577|
|   1.0|  733.0|   4.0|964982400| 3.9428258|
|   1.0| 3479.0|   4.0|964981725| 3.8731346|
|   1.0| 2797.0|   4.0|964981710| 4.5879683|
|   1.0|  804.0|   4.0|964980499| 4.2076497|
|   1.0|  

In [0]:
# recommend top 10 movies for each user:
user_recs = best_model.recommendForAllUsers(10)

In [0]:
def get_recs_for_user(recs,user_id):
  # recs should be for a specific user
  recs = recs.filter(recs.userId == user_id)
  recs = recs.select("recommendations.movieId","recommendations.rating")
  movies = recs.select("movieId").toPandas().iloc[0,0]
  ratings = recs.select("rating").toPandas().iloc[0,0]
  ratings_matrix = pd.DataFrame(movies, columns = ["movieId"])
  ratings_matrix["ratings"] = ratings
  ratings_matrix_ps = spark.createDataFrame(ratings_matrix)
  return ratings_matrix_ps

In [102]:
get_recs_for_user(user_recs, 472).show()

+-------+------------------+
|movieId|           ratings|
+-------+------------------+
|  68945| 5.195949554443359|
|   7841| 5.105589389801025|
|  59018| 5.105589389801025|
|   7481| 5.105589389801025|
|  42730| 5.080193519592285|
|  33649|5.0477519035339355|
| 177593| 5.040066242218018|
|  67618| 5.033014297485352|
|  87234| 5.006368160247803|
|   5328| 4.939907550811768|
+-------+------------------+



In [0]:
# result = df.filter(df.age == 30).collect()
user472 = user_recs.filter(user_recs.userId == 472)

In [76]:
user_recs

DataFrame[userId: int, recommendations: array<struct<movieId:int,rating:float>>]

In [89]:
type(user472)

pyspark.sql.dataframe.DataFrame

In [94]:
 user472.select("recommendations.movieId","recommendations.rating").select("movieId").toPandas().iloc[0,0]

[68945, 7841, 59018, 7481, 42730, 33649, 177593, 67618, 87234, 5328]

In [0]:
#convert list to rdd
rdd = spark.sparkContext.parallelize(user472)

In [86]:
rdd.collect()

[Row(userId=472, recommendations=[Row(movieId=68945, rating=5.195949554443359), Row(movieId=7841, rating=5.105589389801025), Row(movieId=59018, rating=5.105589389801025), Row(movieId=7481, rating=5.105589389801025), Row(movieId=42730, rating=5.080193519592285), Row(movieId=33649, rating=5.0477519035339355), Row(movieId=177593, rating=5.040066242218018), Row(movieId=67618, rating=5.033014297485352), Row(movieId=87234, rating=5.006368160247803), Row(movieId=5328, rating=4.939907550811768)])]

In [83]:
df = spark.createDataFrame(rdd)
print(df.schema)

StructType(List(StructField(userId,LongType,true),StructField(recommendations,ArrayType(StructType(List(StructField(movieId,LongType,true),StructField(rating,DoubleType,true))),true),true)))


In [84]:
# Create a schema for the dataframe
schema = StructType([
    StructField('Category', StringType(), True),
    StructField('Count', DecimalType(), True),
    StructField('Description', StringType(), True)
])


<bound method DataFrame.printSchema of DataFrame[userId: bigint, recommendations: array<struct<movieId:bigint,rating:double>>]>

In [78]:
(user472.select("recommendations.movieId","recommendations.rating").show()

AttributeError: ignored

In [0]:
user_472 = user_recs

In [71]:
user_recs.sort("userId").show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[[68945, 5.8711],...|
|     2|[[131724, 4.81852...|
|     3|[[70946, 4.903539...|
|     4|[[6375, 5.167196]...|
|     5|[[68945, 4.920695...|
|     6|[[67618, 5.114938...|
|     7|[[8477, 4.859085]...|
|     8|[[68945, 5.091939...|
|     9|[[68945, 5.057496...|
|    10|[[112804, 4.65248...|
|    11|[[68945, 5.374137...|
|    12|[[32892, 5.910976...|
|    13|[[68945, 5.369232...|
|    14|[[68945, 4.603229...|
|    15|[[177593, 4.43320...|
|    16|[[68945, 4.648854...|
|    17|[[68945, 5.289617...|
|    18|[[68945, 5.054741...|
|    19|[[68945, 4.228153...|
|    20|[[5490, 5.2965474...|
+------+--------------------+
only showing top 20 rows

